In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
!pip install --upgrade xlrd #colab bug verson xlrd
!pip install geopandas
!pip install pandas fiona shapely pyproj rtree
!pip install contextily
!pip install pygeos
!pip install topojson
import geopandas as gpd
import contextily as ctx
import matplotlib.pyplot as plt
#!pip install requests py7zr geopandas openpyxl tqdm s3fs PyYAML xlrd
#!pip install git+https://github.com/inseefrlab/cartiflette@80b8a5a28371feb6df31d55bcc2617948a5f9b1a

In [2]:
#test_1 = pd.read_csv("https://www.data.gouv.fr/fr/datasets/r/5f71ba43-afc8-43a0-b306-dafe29940f9c", sep = ";", encoding = 'utf-8', engine = 'python')
#le lien du dossier data.gouv : 
# https://www.data.gouv.fr/fr/datasets/effectif-de-patients-par-pathologie-sexe-classe-dage-et-territoire-departement-region/
# methode "classique" pour obtenir un dataset a partir d un lien 
# probleme d encodage d ou les parametres sep, encoding, engine
# A FAIRE : COPIER LA BASE DE FACON LOCALE ET S Y REFERER PAR LA SUITE PAR EFFECTIFS.CSV 

In [3]:
test_2 = pd.read_csv("effectifs.csv", sep = ";", encoding = 'utf-8', engine = 'python')
#commande a utiliser une fois qu on a la base 
#copiee en local 
#la base au prealable telechargee sur ordi perso

In [4]:
test_2.head()
test_2.shape

(4057201, 16)

In [5]:
print("nombre de categories differentes pour niv2 est :"+str(len(test_2['patho_niv2'].unique())))
print("nombre de categories differentes pour niv3 est :"+str(len(test_2['patho_niv3'].unique())))

nombre de categories differentes pour niv2 est :46
nombre de categories differentes pour niv3 est :59


In [6]:
#potentiellement creer 123 colonnes pour pouvoir faire du clustering 
#comparer les clusters obtenus par les caracteristiques d un inidivu pour voir si ca matche ou pas 
print(test_2['patho_niv1'].unique())

['Pas de pathologies repérées, traitements, maternité, hospitalisations ni traitement antalgique ou anti-inflammatoire'
 'Séjours en hospitalisation complète pour prise en charge de la Covid-19'
 'Traitements du risque vasculaire (hors pathologies)'
 'Maladies inflammatoires ou rares ou VIH ou SIDA'
 'Maladies neurologiques ou dégénératives'
 'Traitements psychotropes (hors pathologies)'
 'Traitement antalgique ou anti-inflammatoire (hors pathologies, traitements, maternité ou hospitalisations)'
 'Maladies psychiatriques' nan
 'Autres affections de longue durée (dont 31 et 32)' 'Cancers'
 'Maladies cardio-neurovasculaires'
 'Maladies du foie ou du pancréas (hors mucoviscidose)' 'Diabète'
 'Hospitalisations ponctuelles (avec ou sans pathologies, traitements ou maternité)'
 'Maladies respiratoires chroniques (hors mucoviscidose)'
 'Maternité (avec ou sans pathologies)'
 'Insuffisance rénale chronique terminale']


In [7]:
data_gen = test_2

In [8]:
print(test_2.shape, data_gen.shape)

(4057201, 16) (4057201, 16)


In [9]:
liste_diff_patho1 = data_gen['patho_niv1'].unique()
liste_diff_patho2 = data_gen['patho_niv2'].unique()
liste_diff_patho3 = data_gen['patho_niv3'].unique()

print(liste_diff_patho1, "\n", liste_diff_patho2, "\n", liste_diff_patho3)

['Pas de pathologies repérées, traitements, maternité, hospitalisations ni traitement antalgique ou anti-inflammatoire'
 'Séjours en hospitalisation complète pour prise en charge de la Covid-19'
 'Traitements du risque vasculaire (hors pathologies)'
 'Maladies inflammatoires ou rares ou VIH ou SIDA'
 'Maladies neurologiques ou dégénératives'
 'Traitements psychotropes (hors pathologies)'
 'Traitement antalgique ou anti-inflammatoire (hors pathologies, traitements, maternité ou hospitalisations)'
 'Maladies psychiatriques' nan
 'Autres affections de longue durée (dont 31 et 32)' 'Cancers'
 'Maladies cardio-neurovasculaires'
 'Maladies du foie ou du pancréas (hors mucoviscidose)' 'Diabète'
 'Hospitalisations ponctuelles (avec ou sans pathologies, traitements ou maternité)'
 'Maladies respiratoires chroniques (hors mucoviscidose)'
 'Maternité (avec ou sans pathologies)'
 'Insuffisance rénale chronique terminale'] 
 [nan
 'Séjours en hospitalisation complète pour prise en charge de la Covi

In [10]:
print(data_gen.shape, data_gen['patho_niv1'].isna().sum())

(4057201, 16) 52921


In [11]:
#Creation d une colonne individus
ind = []
for i in range (1,4057202) : 
    ind.append(i)

In [12]:
ind = pd.DataFrame(ind)
ind = ind.set_axis(['ind'], axis='columns')

In [13]:
data_gen_1 = pd.concat([ind, data_gen], axis = 1)

In [14]:
data_gen_1.head()

,ind,annee,patho_niv1,patho_niv2,patho_niv3,top,cla_age_5,sexe,region,dept,ntop,npop,prev,niveau_prioritaire,libelle_classe_age,libelle_sexe,tri
0,1,2017.0,"Pas de pathologies repérées, traitements, mate...",NaN,NaN,sup_Petit_Conso_exclu,45-49,9.0,75,999,230570.0,393250.0,58.632,1,de 45 à 49 ans,tous sexes,17.0
1,2,2017.0,"Pas de pathologies repérées, traitements, mate...",NaN,NaN,sup_Petit_Conso_exclu,45-49,9.0,76,11,14220.0,24330.0,58.451,1,de 45 à 49 ans,tous sexes,17.0
2,3,2017.0,"Pas de pathologies repérées, traitements, mate...",NaN,NaN,sup_Petit_Conso_exclu,45-49,9.0,76,12,10630.0,17390.0,61.133,1,de 45 à 49 ans,tous sexes,17.0
3,4,2017.0,"Pas de pathologies repérées, traitements, mate...",NaN,NaN,sup_Petit_Conso_exclu,45-49,9.0,76,30,30760.0,50820.0,60.539,1,de 45 à 49 ans,tous sexes,17.0
4,5,2017.0,"Pas de pathologies repérées, traitements, mate...",NaN,NaN,sup_Petit_Conso_exclu,45-49,9.0,76,48,3000.0,4920.0,60.898,1,de 45 à 49 ans,tous sexes,17.0


In [15]:
data_gen = data_gen_1

In [16]:
#tentative de compréhension de ce que la variable npop représente
#b = data_gen['npop'].unique()
#b = pd.DataFrame(b)

In [17]:
#b = b.dropna()

In [18]:
#b = b.set_axis(['test'], axis = 'columns')

In [19]:
#b.head()

In [20]:
#sum(b['test'])

In [21]:
#resultat peu concluant

In [22]:
#creation d indicatrices pour futur clustering

In [23]:
liste_diff_patho3 = list(liste_diff_patho3)

In [24]:
#premier code pour creation d indicatrice

In [25]:
temporaire = []
for i in data_gen['patho_niv3'] : 
    if i == 'Syndrome coronaire aigu' :
        temporaire.append(1)
    else : 
        temporaire.append(0)

data_gen['Syndrome_coronaire_aigu_indicatrice'] = temporaire
data_gen = data_gen.drop(columns = ['Syndrome_coronaire_aigu_indicatrice'])
data_gen.head()

,ind,annee,patho_niv1,patho_niv2,patho_niv3,top,cla_age_5,sexe,region,dept,ntop,npop,prev,niveau_prioritaire,libelle_classe_age,libelle_sexe,tri
0,1,2017.0,"Pas de pathologies repérées, traitements, mate...",NaN,NaN,sup_Petit_Conso_exclu,45-49,9.0,75,999,230570.0,393250.0,58.632,1,de 45 à 49 ans,tous sexes,17.0
1,2,2017.0,"Pas de pathologies repérées, traitements, mate...",NaN,NaN,sup_Petit_Conso_exclu,45-49,9.0,76,11,14220.0,24330.0,58.451,1,de 45 à 49 ans,tous sexes,17.0
2,3,2017.0,"Pas de pathologies repérées, traitements, mate...",NaN,NaN,sup_Petit_Conso_exclu,45-49,9.0,76,12,10630.0,17390.0,61.133,1,de 45 à 49 ans,tous sexes,17.0
3,4,2017.0,"Pas de pathologies repérées, traitements, mate...",NaN,NaN,sup_Petit_Conso_exclu,45-49,9.0,76,30,30760.0,50820.0,60.539,1,de 45 à 49 ans,tous sexes,17.0
4,5,2017.0,"Pas de pathologies repérées, traitements, mate...",NaN,NaN,sup_Petit_Conso_exclu,45-49,9.0,76,48,3000.0,4920.0,60.898,1,de 45 à 49 ans,tous sexes,17.0


In [26]:
def creation_indicatrice (colonne_de_depart, nom_de_maladie) : 
    temporaire = []
    for i in colonne_de_depart :
        if i == f"{nom_de_maladie}" :
            temporaire.append(1)
        else :
            temporaire.append(0)
    data_gen[f"{nom_de_maladie}_ind"] = temporaire

In [27]:
for i in liste_diff_patho3 : 
    creation_indicatrice(colonne_de_depart = data_gen['patho_niv3'], nom_de_maladie = i)

In [28]:
#perte d un grand nombre de lignes 
#pb a regler
#pb regle, venait du test pour essayer de comprendre la variable npop

In [29]:
liste_var = data_gen.columns[17:]
liste_bis = []
for i in liste_var : 
    liste_bis.append(i)


In [30]:
indicatrice_patho_niv3 = liste_bis

In [31]:
liste_auxil = data_gen['cla_age_5'].unique()

In [32]:
#afin de pouvoir clusteriser par la suite, nous allons creer des classes d age qui sont "discretes"
#nous faisons ce choix car les classes d age sont relativement homogene et en prenant la moyenne des deux 
#bornes pour les classes, nous gardons l idee d une distance d age qui pourrait etre utile par la suite

In [33]:
test_de_split = data_gen['cla_age_5'][1].split('-')
print(test_de_split)
test_de_split[1] = int(test_de_split[1])
type(test_de_split[1])

['45', '49']


int

In [34]:
inter_excl = ('95et+', 'tsage', np.nan)

inter_def = []
for i in liste_auxil :
    if i in inter_excl : 
        inter_def = inter_def
    else : 
        inter_def.append(i)


In [35]:
print(inter_def)

['45-49', '50-54', '55-59', '60-64', '65-69', '70-74', '75-79', '80-84', '85-89', '90-94', '00-04', '05-09', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40-44']


In [36]:
classe_age = []
for i in data_gen['cla_age_5'] : 
    if i in inter_def :
        temp = i.split('-')
        temp[0] = int(temp[0])
        temp[1] = int(temp[1])
        age = (temp[1] + temp[0])/2
    elif i == 'tsage' : 
        age = np.nan
    elif i == '95et+' : 
        age = 100
    else : 
        age = np.nan
    classe_age.append(age)

In [37]:
#pb a regler, il manque un truc pour eviter que ca bug fort
#probablement type a regler ou categorie a revoir
#code peu ergonomique mais solution la plus sure, 
#toutes les autres tentatives en posant des conditions sur le type de i 
#n ont pas ete solde par une reussite
#on a print nombre de valeurs de classe_age et le code fonctionne

In [38]:
data_gen['classe_age'] = classe_age

In [39]:
data_patho_niv3 = data_gen

In [40]:
data_gen = data_gen.drop(indicatrice_patho_niv3, axis = 1)

In [41]:
#on cree un nouveau dataframe pour pouvoir faire des manipulations plus rapides sur le data_gen

In [42]:
data_gen.head()

,ind,annee,patho_niv1,patho_niv2,patho_niv3,top,cla_age_5,sexe,region,dept,ntop,npop,prev,niveau_prioritaire,libelle_classe_age,libelle_sexe,tri,classe_age
0,1,2017.0,"Pas de pathologies repérées, traitements, mate...",NaN,NaN,sup_Petit_Conso_exclu,45-49,9.0,75,999,230570.0,393250.0,58.632,1,de 45 à 49 ans,tous sexes,17.0,47.0
1,2,2017.0,"Pas de pathologies repérées, traitements, mate...",NaN,NaN,sup_Petit_Conso_exclu,45-49,9.0,76,11,14220.0,24330.0,58.451,1,de 45 à 49 ans,tous sexes,17.0,47.0
2,3,2017.0,"Pas de pathologies repérées, traitements, mate...",NaN,NaN,sup_Petit_Conso_exclu,45-49,9.0,76,12,10630.0,17390.0,61.133,1,de 45 à 49 ans,tous sexes,17.0,47.0
3,4,2017.0,"Pas de pathologies repérées, traitements, mate...",NaN,NaN,sup_Petit_Conso_exclu,45-49,9.0,76,30,30760.0,50820.0,60.539,1,de 45 à 49 ans,tous sexes,17.0,47.0
4,5,2017.0,"Pas de pathologies repérées, traitements, mate...",NaN,NaN,sup_Petit_Conso_exclu,45-49,9.0,76,48,3000.0,4920.0,60.898,1,de 45 à 49 ans,tous sexes,17.0,47.0


In [43]:
df_model = data_gen[['ind','classe_age' , 'region', 'dept']]

In [44]:
df_model_niv3 = pd.concat([df_model, data_patho_niv3[indicatrice_patho_niv3]], axis = 1)

In [45]:
#truc bizarre a reverifier mais il se peut qu'il y ait eu un petit bug avec une colonne nan_ind

In [46]:
df_model_niv3.head()

,ind,classe_age,region,dept,nan_ind,Séjours en hospitalisation complète pour prise en charge de la Covid-19_ind,Traitements antihypertenseurs (hors pathologies)_ind,VIH ou SIDA_ind,Autres affections neurologiques_ind,Traitements antidépresseurs ou régulateurs de l'humeur (hors pathologies)_ind,...,Cancer du sein de la femme actif_ind,Dialyse chronique_ind,Transplantation rénale_ind,Cancer du poumon sous surveillance_ind,Cancer du poumon actif_ind,Suivi de transplantation rénale_ind,Cancer de la prostate sous surveillance_ind,Cancer colorectal actif_ind,Cancer colorectal sous surveillance_ind,Cancer de la prostate actif_ind
0,1,47.0,75,999,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,47.0,76,11,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,47.0,76,12,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,47.0,76,30,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,47.0,76,48,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [47]:
len(df_model['region'].unique())

20

In [48]:
print(df_model['region'].unique())

[75 76 84 93 94  1  2  3  6 11 24 27 28 32 44 52 53 99  4  0]


In [49]:
len(df_model['dept'].unique())

103

In [50]:
df_model['dept'].unique()

array(['999', '11', '12', '30', '48', '81', '82', '09', '01', '42', '74',
       '13', '05', '2A', '971', '972', '976', '75', '91', '18', '28',
       '41', '21', '71', '14', '76', '62', '80', '55', '57', '68', '53',
       '72', '85', '22', '35', '56', '24', '86', '32', '34', '46', '15',
       '07', '06', '2B', '974', '77', '92', '93', '36', '25', '50', '61',
       '59', '60', '52', '54', '67', '08', '16', '64', '79', '87', '31',
       '66', '43', '83', '84', '90', '10', '51', '88', '29', '33', '65',
       '26', '73', '94', '95', '45', '70', '44', '19', '40', '47', '38',
       '63', '89', '17', '04', '973', '39', '58', '27', '02', '69', '49',
       '78', '37', '03', '23', '00'], dtype=object)

In [51]:
#l'autre base pour la pollution ne prend pas les départements d'outre-mer en compte
#il est donc probable que l'on soit contraints de les exclure de l'analyse 


In [52]:
data_pollution = pd.read_csv("base_pollution.csv")

In [53]:
data_pollution.head()

,INSEE commune,Commune,Agriculture,Autres transports,Autres transports international,CO2 biomasse hors-total,Déchets,Energie,Industrie hors-énergie,Résidentiel,Routier,Tertiaire,REG,DEP
0,01001,L'ABERGEMENT-CLEMENCIAT,3711.425991,NaN,NaN,432.751835,101.430476,2.354558,6.911213,309.358195,793.156501,367.036172,84.0,01
1,01002,L'ABERGEMENT-DE-VAREY,475.330205,NaN,NaN,140.741660,140.675439,2.354558,6.911213,104.866444,348.997893,112.934207,84.0,01
2,01004,AMBERIEU-EN-BUGEY,499.043526,212.577908,NaN,10313.446515,5314.314445,998.332482,2930.354461,16616.822534,15642.420313,10732.376934,84.0,01
3,01005,AMBERIEUX-EN-DOMBES,1859.160954,NaN,NaN,1144.429311,216.217508,94.182310,276.448534,663.683146,1756.341319,782.404357,84.0,01
4,01006,AMBLEON,448.966808,NaN,NaN,77.033834,48.401549,NaN,NaN,43.714019,398.786800,51.681756,84.0,01


In [ ]:
#df_model_test = pd.merge(df_model, data_pollution, how = 'left', left_on='dept', right_on='DEP')